## 14.2 ポアソン回帰
```statsmodels```と```sklearn```を用いたポアソン回帰のスニペット。

```sklearn```のエラーを解消できていない。

### (共通)データの読み込み
```seaborn```の```tips```データセットを用いる。

In [ ]:
import polars as pl
pl.Config.set_tbl_cols(-1)
import seaborn as sns

# データの読み込み
acs = pl.read_csv("../data/acs_ny.csv")
acs_subset = acs[["NumBedrooms", "HouseCosts", "OwnRent"]].sort(by = "OwnRent")
display(acs_subset.head())

# 
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
sns.countplot(data = acs, x = "NumBedrooms", ax = ax)

ax.set_title("Number of Bedrooms")
ax.set_xlabel("Number of Bedrooms in a House")
ax.set_ylabel("Count")

plt.show()

### 14.1.1 statsmodels

In [ ]:
import statsmodels.formula.api as smf

# モデルを学習する
form = "NumBedrooms ~ HouseCosts + OwnRent"
model = smf.poisson(formula = form, data = acs_subset).fit()

# 学習結果を表示する
display(model.summary())

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 一般化線形モデルの場合
# モデルを学習する
form = "NumBedrooms ~ HouseCosts + OwnRent"
model = (
    smf.glm(
        formula = form,
        data = acs_subset,
        family = sm.families.Poisson(sm.genmod.families.links.log())
    ).fit()
)

# 学習結果を表示する
display(model.summary())

### 14.1.2 sklearn

In [ ]:
from sklearn import linear_model

# ダミー変数を作成する
import polars.selectors as cs
# 数値型は外しておく
df_numeric = acs_subset.select(cs.numeric()).drop("NumBedrooms")
# カテゴリ変数のみダミー変数を作成する
df_categorical = acs_subset.select(cs.string())
df_dummy = df_categorical.to_dummies(drop_first = True)
# 学習用のDataFrameを作成する
df_study = pl.concat(items = [df_numeric, df_dummy], how = "horizontal")
display(df_study.head())

# モデルを学習する
lr = linear_model.PoissonRegressor()
model = lr.fit(X = df_study, y = acs_subset["NumBedrooms"])


# 学習結果を表示する
import numpy as np
# 係数の名称
labels = ["Intercept"]
for col in df_study.columns:
    labels.append(col)
# 係数
coefficients = np.append(
    model.intercept_, model.coef_
)
# 学習結果
result = pl.DataFrame({
    "Label": labels,
    "Coefficient": coefficients
})
print(result)